In [ ]:
pip install sagemaker -U

In [ ]:
import sagemaker
print(sagemaker.__version__)

In [ ]:
!pygmentize ./src/1/hello_sagemaker_training.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='1.9.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.mxnet import MXNet
estimator = MXNet(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py37', 
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py3', 
    framework_version='0.23-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
estimator.latest_training_job.describe()

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    image_uri = estimator.latest_training_job.describe()['AlgorithmSpecification']['TrainingImage'],
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
with open('./data/1-2/calc.txt','rt') as f:
    input_text_lines = f.read()
for input_text in input_text_lines.split('\n'):
    print(eval(input_text))

In [ ]:
import sagemaker
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2/calc.txt', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2')
print(input_s3_uri)

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-2/calc.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-3/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-3')
print(input_s3_uri)
estimator = TensorFlow(
    entry_point='./src/1-3/calc.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
fold1_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-4/fold1/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-4/fold1')
fold2_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-4/fold2/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-4/fold2')
print(fold1_input_s3_uri, fold2_input_s3_uri)
estimator = TensorFlow(
    entry_point='./src/1-4/calc.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'fold1':fold1_input_s3_uri,
    'fold2':fold2_input_s3_uri,
})

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-5/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-5')
estimator = TensorFlow(
    entry_point='./src/1-5/concat.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

estimator.latest_training_job.describe()

import tarfile
prefix = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts'].replace(f's3://{sagemaker.session.Session().default_bucket()}/','')
sagemaker.session.Session().download_data('./', sagemaker.session.Session().default_bucket(), key_prefix=prefix)
with tarfile.open('./model.tar.gz', 'r') as f:
    f.extractall()

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-5/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-5')
estimator = TensorFlow(
    base_job_name = 'concat', # 追加
    output_path = f's3://{sagemaker.session.Session().default_bucket()}/concat-job', # 追加
    entry_point='./src/1-5/concat.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

estimator.latest_training_job.describe()

import tarfile
prefix = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts'].replace(f's3://{sagemaker.session.Session().default_bucket()}/','')
sagemaker.session.Session().download_data('./', sagemaker.session.Session().default_bucket(), key_prefix=prefix)
with tarfile.open('./model.tar.gz', 'r') as f:
    f.extractall()

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='bf4_version_check.py',
    source_dir='./src/1-6',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-7/switch_process_by_hyperparameter.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'first-num':5,
        'second-num':2,
        'operator':'m'
    }
)
estimator.fit()

In [ ]:
import boto3
logs = boto3.client('logs')
log_group_name = '/aws/sagemaker/TrainingJobs'
latest_logstream_name = logs.describe_log_streams(
    logGroupName=log_group_name,
    orderBy='LastEventTime',
    descending=True
)['logStreams'][0]['logStreamName']
logs.get_log_events(
    logGroupName=log_group_name,
    logStreamName=latest_logstream_name,
)